# Extract notes from Zotero references

In [3]:
import pandas as pd
import config
import zotnote as zn

In [2]:
zn.exportNotes(collection = "M8N2VMAP", 
    library_id = config.library_id, api_key = config.api_key)

In [4]:
pd.read_csv("notes.csv")

,id,citation,tags,title,research_question,data,methods,conclusions,the_good,limitations
0,RGDC9R9R,Abadi et al. 2017,Must read,Penetrance of Polygenic Obesity Susceptibility...,Are the effects of obesity-susceptibility loci...,"37 BMI-associated SNPs in 75,230 adults of Eur...",Quantile regression and meta-regression across...,The effects of nine SNPs increased significant...,NaN,NaN
1,U4MNK7QT,Walter et al. 2016,"BMI, Polygenic scores",Association of a Genetic Risk Score With Body ...,Estimates interaction effects between PGS of o...,HRS,Uses a HLM whereby they estimate effects of ag...,NaN,NaN,Pays little attention to selection due to surv...
